In [2]:
%pip install unidecode

   ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.5 kB ? eta -:--:--
   ------ -------------------------------- 41.0/235.5 kB 487.6 kB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 2.3 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 235.5/235.5 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,initcap,month,upper,lower,initcap,trim,regexp_replace
from pyspark.sql import Window
from unidecode import unidecode
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re


###############################################
####  AJUSTE CIUDAD VENUE

# Limpieza de texto
def limpiar_texto(texto):
    if isinstance(texto, str):  # Comprueba si la dirección es una cadena de texto
        # Mayuscula
        texto_limpio = str.upper(texto)
        # Remover tildes y caracteres especiales de entonacion
        texto_limpio = unidecode(texto_limpio)
        return texto_limpio
    else:
        return texto
    
# Register User Defined Function (UDF) con Spark
# udf: Es una función de PySpark que se utiliza para registrar una función definida por el usuario o UDF. Esta función toma dos argumentos: la función definida por el usuario y el tipo de datos de salida del UDF.
# StringType() Especifica el tipo de datos de salida del UDF.
# Si no registras tu función como un UDF, no podrás aplicarla directamente a una columna en un DataFrame de PySpark
limpiar_texto = udf(limpiar_texto, StringType())


#################################################################################
####  FUNCION QUE REEMPLAZA ALGUNAS VERSIONES DE CIUDAD

def replace_cities_aliases(ciudad):
    if isinstance(ciudad, str):  # Comprueba si la dirección es una cadena de texto
        bogota_aliases = ['BOTOTA','BTA','BOGTA','BOG','BOOGTA','BGTA','KENEDY','BGA','CUNDINAMARCA','BGOOTA','BOOTA','BOFOTA','BOGITA','CEDRITOS','BOBOTA','NOGOTA','BOGATA','BOHOTA','BOGPTA','CIUDAD VERDE','CIUDADVERDE','TINTAL','TUSAQUILLO','VEINTE DE JULIO','USAQUEN','BGOTA','BOGOT','HAYUELOS','SALITRE','KENNEDY','PUENTEARANDA','LA CANDELARIA','LOS MARTIRES','BARRIOS UNIDOS','CHAPINERO','DOCE DE OCTUBRE','TEUSAQUILLO','LA CANDELARIA','CANDELARIA','SUBA','CENTRO HISTORICO','CHICO','SAN DIEGO','CENTRO','SAN PATRICIO','SANTA FE']
        barranquilla_aliases = ['ATLANTICO','BAQ','BQLLLA','BQ','BQA','BQLLA','BARRAQNUILLA','BARRANUILLA','BQUILLLA','ER']
        cali_aliases = ['VALLE DEL CAUCA','PARQUE DEL INGENIO','GBV','SAN NICOLAS','SANTA TERESITA']
        cartagena_aliases = ['CTG','GETSEMANI','130001','CALLE DEL SANTISIMO','BARRIO SAN DIEGO']
        ibague_aliases = ['IABGUE']
        medellin_aliases = ['ANTIOQUIA','POBLADO','MED','MDELLIN','POBLADO MEDELLIN','ARANJUEZ','BARRIO COLOMBIA MEDELLIN','GFBF','CENTRO DE MEDELLIN','LA MACARENA']
        pereira_aliases = ['LA BADEA','VILLA OLIMPICA']
        cucuta_aliases = ['NORTE DE SANTANDER','LA MERCED']
        vcencio_aliases = ['VILLAVICENCIO - META','VILAVICENCIO','BARRIO NUEVO MAIZARO'] 
        quibdo_aliases = ['QUIBDO-CHOO'] 
        manizales_aliases = ['SDVF','CHIPRE','PALERMO'] 
        chia_aliases = ['VSDFVS','CENTRO CHIA']
        santa_marta_aliases = ['GBFG'] 
        bmanga_aliases = ['HNB','CALLE CIEN','GIRON','INDUSTRIAL GIRON'] 
        neiva_aliases = ['VILLA OLIMPICA'] 
        valledupar_aliases = ['VALLEUPAR'] 
        if ciudad in bogota_aliases:
            return 'BOGOTA'
        elif ciudad in barranquilla_aliases:
            return 'BARRANQUILLA'
        elif ciudad in cali_aliases:
            return 'CALI'
        elif ciudad in cartagena_aliases:
            return 'CARTAGENA'
        elif ciudad in medellin_aliases:
            return 'MEDELLIN'
        elif ciudad in ibague_aliases:
            return 'IBAGUE'
        elif ciudad in pereira_aliases:
            return 'PEREIRA'
        elif ciudad in cucuta_aliases:
            return 'CUCUTA'
        elif ciudad in vcencio_aliases:
            return 'VILLAVICENCIO'
        elif ciudad in quibdo_aliases:
            return 'QUIBDO'
        elif ciudad in manizales_aliases:
            return 'MANIZALES'
        elif ciudad in chia_aliases:
            return 'CHIA'
        elif ciudad in santa_marta_aliases:
            return 'SANTA MARTA'
        elif ciudad in bmanga_aliases:
            return 'BUCARAMANGA'
        elif ciudad in neiva_aliases:
            return 'NEIVA'
        elif ciudad in valledupar_aliases:
            return 'VALLEDUPAR'
        else:
            return ciudad
    else:
        return ciudad
    
replace_cities_aliases = udf(replace_cities_aliases, StringType())

In [ ]:
## Ventas 2019
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2019/Ds_S3_Sales_2019.parquet/'
sales_2019 = spark.read.parquet(inpath)
#print(sales_2019.count())
#sales_2019.createOrReplaceTempView("sales_2019_vw")

## Ventas 2020
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2020/Ds_S3_Sales_2020.parquet/'
sales_2020 = spark.read.parquet(inpath)
#print(sales_2020.count())
#sales_2020.createOrReplaceTempView("sales_2020_vw")

## Ventas 2021
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2021/Ds_S3_Sales_2021.parquet/'
sales_2021 = spark.read.parquet(inpath)
#print(sales_2021.count())
#sales_2021.createOrReplaceTempView("sales_2021_vw")

## Ventas 2022
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2022/Ds_S3_Sales_2022.parquet/'
sales_2022 = spark.read.parquet(inpath)
#print(sales_2022.count()) 
#sales_2022.createOrReplaceTempView("sales_2022_vw")

# Ventas 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2023/Ds_S3_Sales_2023.parquet/'
sales_2023 = spark.read.parquet(inpath)
#print(sales_2023.count())
#sales_2023.createOrReplaceTempView("sales_2023_vw")


## Ventas 2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/2024/S3/Ds_S3_Sales_2024.parquet/'
sales_2024 = spark.read.parquet(inpath)
#print(sales_2024.count())
#sales_2023.createOrReplaceTempView("sales_2023_vw")

In [ ]:
columnas = ["ORGANIZATION","T_ORGANIZ_ID","PRODUCT_FAMILY","PRODUCT","INVOICE_CONTACT_FMT_NAME", "T_PRODUCT_ID","LICENCE_NUMBER","PRODUCT_STATE","SITE","SEASON","TOPIC","SUB_TOPIC","REFERENCE_DATE","ORDER_DATE_MM_YY","PRODUCT_DATE_TIME","PRODUCT_TIME","FILE_NUMBER","T_ORDER_CONTACT_ID","PROMOTION","AUDIENCE_CATEGORY","AUDIENCE_SUB_CATEGORY","LOGICAL_SEAT_CATEGORY","PERFORMANCE_QUOTA","SALES_CHANNEL","SALES_CHANNEL_N","SALES_CHANNEL_TYPE","UNIT_AMT_ITX","NET_SOLD_T_QTY","NET_SOLD_C_QTY","NET_SOLD_TKT_AMT_ITX","TEAM_1","TEAM_2","ORDER_NUMBER","CHARGE_AMT_ITX","ORDER_CREATION_DATE","ORDER_CONTACT_NUMBER","ORDER_DATE","PRODUCT_DATE","CHARGE_AMT_ETX","total_unit_amt_itx","DN_QUOTA","T_INVOICE_CONTACT_ID","File_state","t_file_state","ORDER_QUANTITY","T_OPERATION_TYPE","PRODUCT_CODE","t_operation_kind","parent_product","INSURED","INSURED_AMT_ITX","T_PERFORMANCE_ID","T_SITE_ID","NET_SOLD_AMT_ITX","PRODUCT_EXTERNAL_NAME","T_SEASON_ID","ACTIVITY","T_ACTIVITY_ID","T_PRODUCT_FAMILY_ID","PRODUCT_TYPE","T_PRODUCT_TYPE_ID","T_FAMILY_TYPE","T_FAMILY_SUB_TYPE","T_TOPIC_ID","T_SHIPMENTFEE_ID","TOWN","TOPIC_KEY","TOPIC_NW","SUB_TOPIC_NW","INSURED_AMT_ETX"]

## Seleccion de columnas ###
sales_2019 = sales_2019.select(columnas)
sales_2020 = sales_2020.select(columnas)
sales_2021 = sales_2021.select(columnas)
sales_2022 = sales_2022.select(columnas)
sales_2023 = sales_2023.select(columnas)
sales_2024 = sales_2024.select(columnas)


# EN 2024 tengo 2023 incluido
#sales = sales_2019.union(sales_2020).union(sales_2021).union(sales_2022).union(sales_2023).union(sales_2024)
sales = sales_2019.union(sales_2020).union(sales_2021).union(sales_2022).union(sales_2023).union(sales_2024)

#print(sales.count())
sales.createOrReplaceTempView("sales_vw")

In [ ]:
# tmp = sales.select('LOGICAL_SEAT_CATEGORY','T_PRODUCT_ID','PRODUCT_FAMILY').filter(col('T_PRODUCT_ID') == 10229437245287).distinct()
# display(tmp)

In [1]:
# TRANSFORMACIONES
# columns_to_concat = ["T_PRODUCT_ID",'PRODUCT_DATE', "PRODUCT_TIME"]
concat_aforo_pases_visita = ["T_PRODUCT_ID",'PRODUCT_DATE', "PRODUCT_TIME"]
concat_aforo_eventos = ["T_PRODUCT_ID",'T_PERFORMANCE_ID','LOGICAL_SEAT_CATEGORY']
concat_eventos = ["T_PRODUCT_ID",'T_PERFORMANCE_ID']
concat_hfee = ['FILE_NUMBER','T_PRODUCT_ID','T_PERFORMANCE_ID']

sales = sales.withColumnRenamed("TOPIC_NW", "TOPIC_N").\
    withColumnRenamed("SUB_TOPIC_NW", "SUB_TOPIC_N").\
    withColumn("PRODUCT", limpiar_texto("PRODUCT")).\
    withColumn("T_PRODUCT_ID", format_string("%.0f", col("T_PRODUCT_ID"))).\
    withColumn("T_PERFORMANCE_ID", format_string("%.0f", col("T_PERFORMANCE_ID"))).\
    withColumn("Fecha_evento", to_timestamp('PRODUCT_DATE_TIME', "MM/dd/yyyy HH:mm:ss")).\
    withColumn("Date_evento", to_date("Fecha_evento")).\
    withColumn("Hora_evento", date_format("PRODUCT_DATE_TIME", "HH:mm:ss")).\
    withColumn("Key_evento", concat_ws("|", *concat_eventos)).\
    withColumn("Key_handling", concat_ws("|", *concat_hfee)).\
    withColumn("LOGICAL_SEAT_CATEGORY", upper("LOGICAL_SEAT_CATEGORY")).\
    withColumn("LOGICAL_SEAT_CATEGORY", regexp_replace(trim(col("LOGICAL_SEAT_CATEGORY")), r'^\s+|\s+$', '')).\
    withColumn("TOWN_N", limpiar_texto("TOWN")).\
    withColumn("TOWN_N", replace_cities_aliases(col('TOWN_N')))
    #withColumn("LOCALIDAD_GP", localidad_agrupada(col("Direccion_new")))


# Aplicación de unificacion de pases de visita y pases con fecha y hora
sales = sales.withColumn("PRODUCT_FAMILY",
                   when(sales["PRODUCT_FAMILY"].isin("Pases con fecha y hora"),
                        lit('Pase de visita')).otherwise(sales["PRODUCT_FAMILY"]))

# # Aplicación de la lógica de reemplazo en el DataFrame df_follwup
sales = sales.withColumn("Key_aforo",
                   when(sales["PRODUCT_FAMILY"].isin("Pase de visita"),
                        concat_ws("|", *concat_aforo_pases_visita)).otherwise(concat_ws("|", *concat_aforo_eventos)))   

sales.createOrReplaceTempView("sales_vw")

NameError: name 'sales' is not defined

In [ ]:
handling = spark.sql(
    """
WITH hfee as (
    SELECT
    s.FILE_NUMBER as Expediente,
    SUM(s.NET_SOLD_TKT_AMT_ITX) as Handling_fee
    FROM sales_vw s
    WHERE s.T_OPERATION_TYPE IN ('SALE','REFUND')
        AND s.T_OPERATION_KIND IN ('DELIVERY_OVERHEADS','ORDER_OVERHEADS')--'COMPOSED_PRODUCT' 
    GROUP BY s.FILE_NUMBER),

eventos as (
    SELECT DISTINCT      
      FILE_NUMBER as Expediente,
      Key_handling
    FROM sales_vw 
    WHERE product_family IN ('Competición','Evento','Pase de visita')
        --AND s.PRODUCT_CODE != 'CAN_INS'
        AND UPPER(PRODUCT) NOT LIKE '%PRUEBA%'
        AND UPPER(PRODUCT) NOT LIKE '%SUITE%'
        AND UPPER(PRODUCT) NOT LIKE '%PARQUEADERO%'
        AND UPPER(PRODUCT) NOT LIKE '%PREIMPRESO%'
        AND UPPER(PRODUCT) NOT LIKE '%ACREDITAC%'
        AND UPPER(PRODUCT) NOT LIKE '%REDENCI%'
        AND UPPER(PRODUCT) NOT LIKE '%ZONAS VIP%'
        AND UPPER(PRODUCT) NOT LIKE '%PCR%'
        AND UPPER(PRODUCT) NOT LIKE '%GENOS%'
        AND UPPER(PRODUCT) NOT LIKE '%TUBOLETA TE LLEVA%'
        AND UPPER(PRODUCT) NOT LIKE '%CONCIERGE%'
        AND UPPER(PRODUCT) NOT LIKE '%VMOVE%'),

n_eventos as (
    SELECT
    Expediente,
    count(Key_handling) as N_eventos_x_expediente
    FROM eventos
    GROUP BY Expediente),

hfee_X_evento as (
    SELECT
    a.Expediente,
    a.N_eventos_x_expediente,
    b.Handling_fee,
    b.Handling_fee/a.N_eventos_x_expediente as Handling_fee_x_evento
    FROM n_eventos a
    LEFT JOIN hfee b
    ON (a.Expediente = b.Expediente)
    WHERE b.Handling_fee != 0 AND b.Handling_fee is not null )

SELECT 
  a.Key_handling,
  b.Handling_fee_x_evento
FROM eventos a
LEFT JOIN hfee_X_evento b
  ON (a.Expediente = b.Expediente)
WHERE (b.Handling_fee_x_evento != 0 AND b.Handling_fee_x_evento is not null)
ORDER BY Key_handling

""")

print(handling.count())
#display(handling.limit(1000))
handling.createOrReplaceTempView("handling_vw")

4272655


In [ ]:
# handling = spark.sql(
#     """
# WITH hfee as (
#     SELECT
#     s.FILE_NUMBER as Expediente,
#     SUM(s.NET_SOLD_TKT_AMT_ITX) as Handling_fee
#     FROM sales_vw s
#     WHERE s.T_OPERATION_TYPE IN ('SALE','REFUND')
#         AND s.T_OPERATION_KIND IN ('DELIVERY_OVERHEADS','ORDER_OVERHEADS')--'COMPOSED_PRODUCT' 
#     GROUP BY s.FILE_NUMBER),

# eventos as (
#     SELECT DISTINCT
#     s.FILE_NUMBER as Expediente,
#     s.Key_evento
#     FROM sales_vw s
#     WHERE s.product_family IN ('Competición','Evento','Pase de visita')
#         --AND s.PRODUCT_CODE != 'CAN_INS'
#         AND UPPER(s.PRODUCT) NOT LIKE '%PRUEBA%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%SUITE%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%PARQUEADERO%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%PREIMPRESO%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%ACREDITAC%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%REDENCI%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%ZONAS VIP%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%PCR%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%GENOS%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%TUBOLETA TE LLEVA%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%CONCIERGE%'
#         AND UPPER(s.PRODUCT) NOT LIKE '%VMOVE%'),

# n_eventos as (
#     SELECT
#     Expediente,
#     count(Key_evento) as N_eventos_x_expediente
#     FROM eventos
#     GROUP BY Expediente)

# SELECT
# a.Expediente,
# a.N_eventos_x_expediente,
# b.Handling_fee,
# b.Handling_fee/a.N_eventos_x_expediente as Handling_fee_x_evento
# FROM n_eventos a
# LEFT JOIN hfee b
#   ON (a.Expediente = b.Expediente)
# WHERE (b.Handling_fee != 0 AND b.Handling_fee is not null)
#     AND a.Expediente = '10000096'


# """)

# #print(handling.count())
# display(handling)


In [ ]:
tmp = sales.select('FILE_NUMBER','PRODUCT','product_family','REFERENCE_DATE','PRODUCT_DATE_TIME').\
    filter(col('FILE_NUMBER') == '10000096').distinct()
display(tmp)

In [ ]:
# tmp = sales.\
#     select('TOWN','SITE','TOWN_V2').distinct()
# # #tmp = sales.filter(col('T_PRODUCT_ID') == '10229196910867') #eladio cali
# # #print(tmp.count())
# display(tmp)

In [ ]:
# # tmp = sales.select('t_file_state').distinct()
# # display(tmp)

# tmp = sales.select('T_PRODUCT_ID','TOPIC').filter(col('T_PRODUCT_ID') == '10229196910867').distinct() #eladio cali
# #print(tmp.count())
# display(tmp)

In [ ]:
# tmp = sales.select('T_PERFORMANCE_ID','TEAM_1','TEAM_2').\
#     filter(col('T_PERFORMANCE_ID') == '10229481277847').distinct()

# #display(tmp.limit(100))
# display(tmp)

In [ ]:
## TABLA AFOROS ACTUALIZADA
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Follwup/Ds_S_Follwup.parquet"
df_aforos = spark.read.parquet(inpath)
print(df_aforos.count())

#df_aforos.createOrReplaceTempView("aforos_vw")

## MAESTRA TOPICS
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Maestro/Maestro_Tema_Subtema/'
maestro = spark.read.parquet(inpath)

print(maestro.count())
#display(maestro.limit(100))


## TABLA AFOROS ACTUALIZADA
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Configuration/Ds_S_Configuration.parquet/'
configuration = spark.read.parquet(inpath)
configuration.createOrReplaceTempView("configuration_vw")
#print(configuration.count())


## ciudad TOWN
blob_storage_path = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Ciudades/Ciudades.csv/"
ciudad = spark.read.csv(blob_storage_path,header=True, inferSchema=True, sep=';')


1839039
25537


In [ ]:
# tmp = df_aforos.filter(col('Key_evento_').like('%10229437245287%'))
# display(tmp)

tmp = configuration.filter(col('Key_evento').like('%10229649694546%'))
display(tmp) 

Key_evento,T_PRODUCT_ID,T_PERFORMANCE_ID,PRODUCT,TEAM_1,TEAM_2,MATCH_DAY,TOPIC,Edad_minima,NIT,Contrato


In [ ]:
# from pyspark.sql.functions import concat_ws
# from pyspark.sql.functions import col

# # Columnas a concatenar
# columns_to_concat = ["ORGANIZATION", "TOWN", "SITE"]

# # Utiliza la función concat_ws para concatenar las columnas con un separador (en este caso, "")
# sales_2 = sales.withColumn("concatenated_column", concat_ws("", *columns_to_concat))


# # Realiza la unión basada en las columnas "concatenated_column" y "CONCATENADO"
# joined_df = sales_2.join(ciudad, sales_2["concatenated_column"] == ciudad["CONCATENADO"], "left_outer")

# # Borras las columnaas del csv
# columns_to_drop = ["ORGANIZATION2","SITE2", "CONCATENADO","concatenated_column"]

# # Crea un nuevo DataFrame sin las columnas especificadas
# result_df = joined_df.drop(*columns_to_drop)


# # Renombra la columna "HOMOLOGA" como "TOWN"
# sales = result_df.withColumnRenamed("HOMOLOGA", "TOWN_V2")

In [ ]:
# CAMPOS DE CADA MAESTRA
venues = ["Emplazamiento_Secutix",'Venue_Panel']
temas_ppto = ["Temas_2023",'Segmento_Presupuesto_2023']
tipo_venue = ["VENUE",'Tipo_de_escenario']
canales_venta = ["CANAL_DE_VENTA",'Agrupacion_Canal']
medios_pago = ["Medio_Pago",'Agrupacion_Medio_Pago']
productos_2024 = ["Producto_2024",'Categoria_Producto']
promotores = ["Promotor_Secutix",'Promotor_Panel']
productos_2023 = ["Producto_2023",'Codigo_de_producto','Sesiones','Aforo','Segmento_Presupuesto_2023_2','Tema','Subtema']

# MAESTRAS
venues = maestro.select(venues).\
    filter(col('Emplazamiento_Secutix').isNotNull())
print(venues.count())
#display(venues)

temas_ppto = maestro.select(temas_ppto).\
    filter(col('Temas_2023').isNotNull())
print(temas_ppto.count())

tipo_venue = maestro.select(tipo_venue).\
    filter(col('VENUE').isNotNull())
print(tipo_venue.count())
#display(tipo_venue)

canales_venta = maestro.select(canales_venta).\
    filter(col('CANAL_DE_VENTA').isNotNull())
print(canales_venta.count())

medios_pago = maestro.select(medios_pago).\
    filter(col('Medio_Pago').isNotNull())
print(medios_pago.count())

productos_2024 = maestro.select(productos_2024).\
    filter(col('Producto_2024').isNotNull())
print(productos_2024.count())

promotores = maestro.select(promotores).\
    filter(col('Promotor_Secutix').isNotNull())
print(promotores.count())

productos_2023 = maestro.select(productos_2023).\
    filter(col('Producto_2023').isNotNull())
print(productos_2023.count())

284
39
271
187
72
823
322
25537


In [ ]:
# AJUSTE DE AFORO
sales = sales.join(df_aforos['Key_evento_',"Nuevo_aforo"],\
    sales["Key_aforo"] == df_aforos["Key_evento_"],\
    "left_outer")


# AJUSTE DE VENUE
sales = sales.join(venues,\
    sales["SITE"] == venues["Emplazamiento_Secutix"],\
    "left_outer").\
    withColumnRenamed("Venue_Panel", "VENUE_N").\
    withColumn("VENUE_N", initcap("VENUE_N"))

# AJUSTE DE TIPO VENUE
sales = sales.join(tipo_venue,\
    sales["VENUE_N"] == tipo_venue["VENUE"],\
    "left_outer").\
    withColumnRenamed("Tipo_de_escenario", "Tipo_venue")

# AJUSTE DE CANALES DE VENTA
# sales = sales.join(canales_venta,\
#     sales["VENUE_N"] == canales_venta["CANAL_DE_VENTA"],\
#     "left_outer").\
#     withColumnRenamed("Tipo_de_escenario", "Tipo_venue")



# # # Crea un nuevo DataFrame sin las columnas especificadas
columns_to_drop = ["Key_evento_",'Emplazamiento_Secutix','VENUE']
sales = sales.drop(*columns_to_drop)


#sales = sales_2021.union(sales_2022).union(sales_2023)

#print(sales_2023.count())
sales.createOrReplaceTempView("sales_vw")
#print(sales.count())

In [ ]:
# Ajuste de equipos de futbol visitantes por definir
sales = spark.sql(
    """
    select 
      s.*,
      CASE 
            WHEN c.TEAM_2 IS NULL THEN s.TEAM_2
            ELSE c.TEAM_2 
        END AS TEAM_2_N,
      c.NIT,
      c.Contrato,
      c.MATCH_DAY,
      CASE 
            WHEN c.TOPIC IS NULL THEN s.TOPIC
            ELSE c.TOPIC 
        END AS TOPIC_PPTO_old,
      CASE 
            WHEN c.PRODUCT IS NULL THEN s.PRODUCT
            ELSE c.PRODUCT 
        END as PRODUCT_N,
      CASE 
            WHEN h.Handling_fee_x_evento IS NULL THEN 0
            ELSE h.Handling_fee_x_evento 
        END as Hfee
    from sales_vw s
    left join configuration_vw c
      on s.Key_evento = c.Key_evento
    left join handling_vw h
      on s.Key_handling = h.Key_handling
    """)

# AJUSTE DE TEMA
sales = sales.join(temas_ppto,\
    sales["TOPIC_PPTO_old"] == temas_ppto["Temas_2023"],\
    "left_outer").\
    withColumnRenamed("Segmento_Presupuesto_2023", "TOPIC_PPTO")


# Borras las columnaas del csv
columns_to_drop = ["TOPIC_PPTO_old",'Temas_2023']
# Crea un nuevo DataFrame sin las columnas especificadas
sales = sales.drop(*columns_to_drop)


#display(sales)
#print(sales.count())

In [ ]:
# tmp = sales.select('Key_evento','Key_aforo','Nuevo_aforo').filter(col('Key_evento').like('%10229437245287%')).distinct()
# display(tmp)

In [ ]:
# tmp = sales.select('T_PERFORMANCE_ID','TEAM_2','TEAM_2_N').\
#     filter(col('T_PERFORMANCE_ID') == '10229481277847').distinct()

# #display(tmp.limit(100))
# display(tmp)

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/'

join_df_coalesced = sales.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)
#join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_sales.parquet")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-608797768202134>, line 6
      3 inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/'
      5 join_df_coalesced = sales.coalesce(1)
----> 6 join_df_coalesced.write.mode("overwrite").parquet(inpath)
      7 #join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)
      8 
      9 #cambio de nombre
     10 files=dbutils.fs.ls(inpath)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1721, in DataFrameWriter.